# Week 4 Assignment for the Capstone Project - Part I: Data Preparation

Developed by Yongkang Liu  
December 27, 2019

<a name="top"></a>  

In the capstone project, I would like to explore the ridership of New York subways and its impact on the selection of coffee shop sites in Manhattan.

In this assignment (Week 4), I will first focus on preparing data for the final assignment. Specifically, data from different sources are collected, cleaned, and formatted. The cleaned data is exported as csv files for future uses.

### Friendly Reminder: If you want to just check the cleaned data, you can skip the data preparation steps and jump to the end of this notebook. [click here](#cleaned) 

In [116]:
# import libraries
import pandas as pd

#### Reference

* [Transportation Data and Examples](http://transitdatatoolkit.com/lessons/mapping-a-transit-system/)

##  Data Source: Turnstile Data
MTA reguarly publishes turnstile data every week. Each file contains information regarding the counts of entries and exits through each turnstile in MTA stations around every 4 hours. Each turnstile is distinguished by UNIT, SCP and STATION. Meanwhile, each station is uniquely identified by the station name, line mark, and division tag.

Major operations on the turnstile data include but not limited to 1) getting the entries and exits through each turntile in each collection period, 2) joining data at the same station and grouping by the date and time, 3) clearning data by removing abnormal or erroneous records.

In [3]:
# Save MTA turnstile data into a dataframe
# Source: http://web.mta.info/developers/turnstile.html
# The file is downloaded and saved in the same folder as the notebook
df_tt = pd.read_csv('turnstile_191102.txt', skipinitialspace=True)  # the data in the week of Nov. 02, 2019
df_tt.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,10/26/2019,00:00:00,REGULAR,7247322,2455491
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,10/26/2019,04:00:00,REGULAR,7247336,2455499
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,10/26/2019,08:00:00,REGULAR,7247351,2455532
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,10/26/2019,12:00:00,REGULAR,7247463,2455623
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,10/26/2019,16:00:00,REGULAR,7247755,2455679


In [4]:
# Check the column headers
print(f'df_tt.columns: {df_tt.columns}')

df_tt.columns: Index(['C/A', 'UNIT', 'SCP', 'STATION', 'LINENAME', 'DIVISION', 'DATE', 'TIME',
       'DESC', 'ENTRIES',
       'EXITS                                                               '],
      dtype='object')


Notice that the last column name, a string variable, contains many space characters. We need to remove them.

In [5]:
print(f'Check the last column name: ("{df_tt.columns[-1]}")')

# rename the column name
df_tt.rename(columns={df_tt.columns[-1]:df_tt.columns[-1].strip(' ')}, inplace=True)
print(f'After the change, the new columns are {df_tt.columns}')

Check the last column name: ("EXITS                                                               ")
After the change, the new columns are Index(['C/A', 'UNIT', 'SCP', 'STATION', 'LINENAME', 'DIVISION', 'DATE', 'TIME',
       'DESC', 'ENTRIES', 'EXITS'],
      dtype='object')


To better identify each station, we design a unique station index. Since a station is uniquely identified by the station name, line name, and division. We use a number string as the label.

In [6]:
print(f'The number of unique station names: {len(df_tt.STATION.unique().tolist())}')
print(f'The number of unique line names: {len(df_tt.LINENAME.unique().tolist())}')
print(f'The number of unique division names: {len(df_tt.DIVISION.unique().tolist())}')

The number of unique station names: 377
The number of unique line names: 113
The number of unique division names: 6


We use three digits to label a station, three digits for a line combination, and one digit for a division. For example, the station "RIT-ROOSEVELT" in Line "R" of Division "RIT" have the encoded indices, "376", "019", and "5", for the station name, line combination, and division, respectively. We label it as "t5019376" in the format of "t-Division-Line-Station" where "t" stands for the turntile data source.

In [7]:
# First, obtain the index in each column
dict_tt_stations = {k: str(v).zfill(3) for v, k in enumerate(df_tt.STATION.unique().tolist())}
dict_tt_lines = {k: str(v).zfill(3) for v, k in enumerate(df_tt.LINENAME.unique().tolist())}
dict_tt_divisions = {k: str(v) for v, k in enumerate(df_tt.DIVISION.unique().tolist())}

# Then, create a new column in the dataframe and assign the unique index 
df_tt['STATION_IDX'] = df_tt[['STATION', 'LINENAME', 'DIVISION']].apply(lambda x: 't'+dict_tt_divisions[x.DIVISION]+dict_tt_lines[x.LINENAME]+dict_tt_stations[x.STATION], axis=1)
df_tt.head()

In [ ]:
# optional: check elements in each column/feature
# Explanations to Terminology can be found in http://web.mta.info/developers/resources/nyct/turnstile/ts_Field_Description.txt
# df_tt.DESC.unique()
# df_tt[df_tt.DESC=='RECOVR AUD']

Notice that the turnstile data does not contain geolcation information, i.e., latitude and longitude, of the station. In order to assign such information to each row of data for the future correlation with Foursquare API, I need additional data in this work. 

## Data Source: MTA Stations' Geolocation Information

I will use the MTA's station geolocation dataset.  

### Station Geo Data

Station geo data includes geographical information of each station in MTA. It mainly provides the location information, i.e., latitude and longitude of each station. The record is mainly distinguished by the station name, line name, division.

The naming rules can be found [here](https://en.wikipedia.org/wiki/New_York_City_Subway_nomenclature).

Note that one station may have multiple exits in different geolocations. As there is no specific location information mapped to each turnstile, I will only consider all turnstiles' data of the same station as one piece in future processes.

First, let us look at the station geolocation dataset.

In [10]:
# There are multiple versions of such data
# Version 1
#df_geo = pd.read_csv('DOITT_SUBWAY_STATION_01_13SEPT2010.csv')
#df_geo.loc[0]['LINE'].split('-')

# Version 2
# df_station_entrances = pd.read_csv('NYC_Transit_Subway_Entrance_And_Exit_Data.csv')

# Version 3
# We are going to use the following data published by MTA
# http://web.mta.info/developers/data/nyct/subway/Stations.csv 
# in GTFS format
df_stations = pd.read_csv('Stations.csv')

df_stations.head()

,Station ID,Complex ID,GTFS Stop ID,Division,Line,Stop Name,Borough,Daytime Routes,Structure,GTFS Latitude,GTFS Longitude,North Direction Label,South Direction Label
0,1,1,R01,BMT,Astoria,Astoria - Ditmars Blvd,Q,N W,Elevated,40.775036,-73.912034,NaN,Manhattan
1,2,2,R03,BMT,Astoria,Astoria Blvd,Q,N W,Elevated,40.770258,-73.917843,Ditmars Blvd,Manhattan
2,3,3,R04,BMT,Astoria,30 Av,Q,N W,Elevated,40.766779,-73.921479,Astoria - Ditmars Blvd,Manhattan
3,4,4,R05,BMT,Astoria,Broadway,Q,N W,Elevated,40.761820,-73.925508,Astoria - Ditmars Blvd,Manhattan
4,5,5,R06,BMT,Astoria,36 Av,Q,N W,Elevated,40.756804,-73.929575,Astoria - Ditmars Blvd,Manhattan


In [12]:
# Remove unrelated columns
df_stations.drop(["Complex ID", "GTFS Stop ID", 'Line', 'Structure', 'North Direction Label', 'South Direction Label'], axis=1, inplace=True)
df_stations.head()

The station data has the unique identifier, i.e., Station ID, for each record. Therefore, we don't need to render a different ID for this dataframe.  

Our next step is to link these Station IDs with the station index created in the turnstile data.

In [14]:
df_stations.Borough.unique().tolist()

['Q', 'M', 'Bk', 'Bx', 'SI']

We further retain our disussion within the Manhatten island. Therefore, we only keep the stations in the "Borough M".

There is another way to retrieve Manhattan-only data, i.e., deleting all rows that contain station data out of Manhattan
```python
indexNames = df_stations[ df_stations['Borough'] != 'M' ].index # first, get indices of these rows
df_stations.drop(indexNames , inplace=True) # remove them from the dataframe
```

In [15]:
df_stations = df_stations[df_stations.Borough=='M']
print(f'We found {df_stations.shape[0]} stations with {df_stations["Stop Name"].nunique()} unique station names')
df_stations.head()

The reason why we have more station records than the unique station names is because one station can have multiple records if it hosts multiple lines.

In [18]:
# The stations in different Routes may have the same name. Their coordinates may differ from each other but all in a vicinity. 
# Since the turntile data uses the stop name for all routes, we will render a coordinate for each hub station 

# check how many records are there for the station name of "Canal St"
df_stations[df_stations['Stop Name']=='Canal St']

,Station ID,Division,Stop Name,Borough,Daytime Routes,GTFS Latitude,GTFS Longitude
17,18,BMT,Canal St,M,R W,40.719527,-74.001775
18,19,BMT,Canal St,M,N Q,40.718383,-74.000460
103,104,BMT,Canal St,M,J Z,40.718092,-73.999892
169,169,IND,Canal St,M,A C E,40.720824,-74.005229
325,325,IRT,Canal St,M,1,40.722854,-74.006277
410,410,IRT,Canal St,M,6,40.718803,-74.000193


It may result in two consequences: 1) one station in the turntile data may have multiple geolocations after the mapping, 2) there may be some wrong mapping due to stations in different lines/divisions share the same name.

To eliminate such errors, I combine division, line, and stop name to uniquely locate a station with the specific Station ID. 

### Matching Stations

In order to find all turnstile data of Manhattan stations and assign them with correct geolocation information, it needs to match the records by the station names in the df_stations.

In [19]:
# Processing the Turnstile dataset

# Since there are only three divisions in Manhattan, I first reduce the turnstile data size 
df_tt[df_tt.DIVISION.isin(['BMT', 'IND', 'IRT'])].STATION.nunique()

# Create a unique search id for each station
df_tt['SEARCH_ID'] = df_tt[['STATION', 'LINENAME', 'DIVISION', 'STATION_IDX']].apply(lambda x: ','.join(x.dropna().astype(str)), axis=1)

In [20]:
# Obtain all stations in the Turnstile dataset and save them into a list
tt_list = df_tt['SEARCH_ID'].unique().tolist()
tt_list = list(map(lambda x: x.split(','), tt_list))

# Convert the list to a dataframe
df_tt_list = pd.DataFrame(tt_list, columns=['Station', 'Routes', 'Division', 'STATION_IDX'])
df_tt_list.head()

# Adjust the name format: change "a-b" to "a - b"
def hyphen_adjust(x):
    if '-' in x:
        tmp = x.split('-')
        return ' - '.join(tmp)
    else:
        return x

#df_tt_list['Station'] = df_tt_list['Station'].apply(hyphen_adjust)
#df_tt_list['Routes'] = df_tt_list['Routes'].apply(lambda x: set(x))

from tqdm import tqdm, tqdm_notebook
tqdm_notebook().pandas()
df_tt_list['Station'] = df_tt_list['Station'].progress_apply(hyphen_adjust)
df_tt_list['Routes'] = df_tt_list['Routes'].progress_apply(lambda x: set(x))
# a progress bar will appear when running the code

df_tt_list.head()

df_tt_list.shape

(482, 4)

In [21]:
df_tt_list.head()

,Station,Routes,Division,STATION_IDX
0,59 ST,"{R, 4, Q, W, N, 5, 6}",BMT,t0000000
1,5 AV/59 ST,"{N, Q, R, W}",BMT,t0001001
2,57 ST - 7 AV,"{N, Q, R, W}",BMT,t0001002
3,49 ST,"{N, Q, R, W}",BMT,t0001003
4,TIMES SQ - 42 ST,"{C, A, 2, R, 1, Q, 7, W, S, N, E, 3}",BMT,t0002004


Next, it is about to obtain the search ID for the geolocation dataset.

In [22]:
# Processing the station profile dataset, i.e., the station geolocation reference
# Obtain unique ID for the qualified stations
df_stations['Search ID'] = df_stations[['Stop Name', 'Daytime Routes', 'Division', 'Station ID']].apply(lambda x: ','.join(x.dropna().astype(str)), axis=1)

df_stations.reset_index(inplace=True)

df_stations.head()

,index,Station ID,Division,Stop Name,Borough,Daytime Routes,GTFS Latitude,GTFS Longitude,Search ID
0,6,7,BMT,Lexington Av/59 St,M,N W R,40.762660,-73.967258,"Lexington Av/59 St,N W R,BMT,7"
1,7,8,BMT,5 Av/59 St,M,N W R,40.764811,-73.973347,"5 Av/59 St,N W R,BMT,8"
2,8,9,BMT,57 St - 7 Av,M,N Q R W,40.764664,-73.980658,"57 St - 7 Av,N Q R W,BMT,9"
3,9,10,BMT,49 St,M,N R W,40.759901,-73.984139,"49 St,N R W,BMT,10"
4,10,11,BMT,Times Sq - 42 St,M,N Q R W,40.754672,-73.986754,"Times Sq - 42 St,N Q R W,BMT,11"


In [23]:
sta_list = df_stations['Search ID'].unique().tolist()

sta_list = list(map(lambda x: x.upper(), sta_list))  # Captalize all names

sta_list = list(map(lambda x: x.split(','), sta_list))

df_station_list = pd.DataFrame(sta_list, columns=['Station', 'Routes', 'Division', 'Station ID'])  # save into a dataframe

df_station_list['Routes'] = df_station_list['Routes'].apply(lambda x: set(x.split(' '))) # split routes into a list

df_station_list.head()

df_station_list.shape

(153, 4)

In [24]:
df_station_list.head()

,Station,Routes,Division,Station ID
0,LEXINGTON AV/59 ST,"{N, R, W}",BMT,7
1,5 AV/59 ST,"{N, R, W}",BMT,8
2,57 ST - 7 AV,"{N, Q, R, W}",BMT,9
3,49 ST,"{N, R, W}",BMT,10
4,TIMES SQ - 42 ST,"{N, Q, R, W}",BMT,11


Now, the job is to find these 153 subway stations in Manhattan in the station list of the Turnstile data 

In [25]:
# Define a new column to save the matched station
df_tt_list['Geo_ID']=df_tt_list['Station'].apply(lambda x: [])

# Define a new column to signal the match result in the reference station dataframe and set initial values to "False" 
df_station_list['Matched']=df_station_list['Station'].apply(lambda x: False)

In [26]:
'''
def station_match(tt, station):
    matched = False
    #print(f'tt: {tt}, station: {station}')
    if tt[2] == station[2]:  # the same division
        if tt[0] == station[0]:  # the same name
            #print(f'Station: {station} matches with TT: {tt}')
            if station[1].issubset(tt[1]):  # route set match
                matched = True
                print(f'Station: {station} matches with TT: {tt} in lines {station[1]}')
    return matched
'''
# a relaxed version which removes the division comparison because some stations in the Turntile data can be a union of multiple stations of different divisions
# The station name plus service routes can adequately define a unique station
def station_match(tt, station):
    matched = False
    #print(f'tt: {tt}, station: {station}')
    if tt[0] == station[0]:
        #print(f'Station: {station} matches with TT: {tt}')
        if station[1].issubset(tt[1]):
            matched = True
            print(f'Station: {station} matches with TT: {tt} in lines {station[1]}')
    return matched

In [27]:
count = 0
dTest = {}
dict_stations = {}
for i in range(df_station_list.shape[0]):
    for j in range(df_tt_list.shape[0]):
        if station_match(list(df_tt_list.loc[j]), list(df_station_list.loc[i])):
            count += 1
            # Add the Station_ID into The turntile record 
            df_tt_list.loc[j]['Geo_ID'].append(df_station_list.at[i, "Station ID"])
            df_station_list.at[i, 'Matched'] = True
            if df_tt_list.loc[j]['STATION_IDX'] in dTest:
                dict_stations[df_tt_list.loc[j]['STATION_IDX']].append(df_station_list.at[i, "Station ID"])
            else:
                dict_stations[df_tt_list.loc[j]['STATION_IDX']] = [df_station_list.at[i, "Station ID"]]

Station: ['5 AV/59 ST', {'N', 'R', 'W'}, 'BMT', '8', False] matches with TT: ['5 AV/59 ST', {'N', 'Q', 'R', 'W'}, 'BMT', 't0001001', []] in lines {'N', 'R', 'W'}
Station: ['57 ST - 7 AV', {'N', 'Q', 'R', 'W'}, 'BMT', '9', False] matches with TT: ['57 ST - 7 AV', {'N', 'Q', 'R', 'W'}, 'BMT', 't0001002', []] in lines {'N', 'Q', 'R', 'W'}
Station: ['49 ST', {'N', 'R', 'W'}, 'BMT', '10', False] matches with TT: ['49 ST', {'N', 'Q', 'R', 'W'}, 'BMT', 't0001003', []] in lines {'N', 'R', 'W'}
Station: ['TIMES SQ - 42 ST', {'N', 'Q', 'R', 'W'}, 'BMT', '11', False] matches with TT: ['TIMES SQ - 42 ST', {'C', 'A', '2', 'R', '1', 'Q', '7', 'W', 'S', 'N', 'E', '3'}, 'BMT', 't0002004', []] in lines {'N', 'Q', 'R', 'W'}
Station: ['TIMES SQ - 42 ST', {'N', 'Q', 'R', 'W'}, 'BMT', '11', True] matches with TT: ['TIMES SQ - 42 ST', {'2', 'A', 'C', 'R', '1', '7', 'Q', 'W', 'S', 'N', 'E', '3'}, 'IRT', 't3085004', []] in lines {'N', 'Q', 'R', 'W'}
Station: ['34 ST - HERALD SQ', {'N', 'Q', 'R', 'W'}, 'BMT', 

Station: ['86 ST', {'4', '6', '5'}, 'IRT', '397', False] matches with TT: ['86 ST', {'4', '6', '5'}, 'IRT', 't3095052', []] in lines {'4', '6', '5'}
Station: ['77 ST', {'6'}, 'IRT', '398', False] matches with TT: ['77 ST', {'6'}, 'IRT', 't3090051', []] in lines {'6'}
Station: ['59 ST', {'4', '6', '5'}, 'IRT', '400', False] matches with TT: ['59 ST', {'R', '4', 'Q', 'W', 'N', '5', '6'}, 'BMT', 't0000000', []] in lines {'4', '6', '5'}
Station: ['59 ST', {'4', '6', '5'}, 'IRT', '400', True] matches with TT: ['59 ST', {'R', '4', 'Q', 'W', 'N', '5', '6'}, 'IRT', 't3094000', []] in lines {'4', '6', '5'}
Station: ['51 ST', {'6'}, 'IRT', '401', False] matches with TT: ['51 ST', {'6'}, 'IRT', 't3090281', []] in lines {'6'}
Station: ['33 ST', {'6'}, 'IRT', '403', False] matches with TT: ['33 ST', {'6'}, 'IRT', 't3090279', []] in lines {'6'}
Station: ['28 ST', {'6'}, 'IRT', '404', False] matches with TT: ['28 ST', {'6'}, 'IRT', 't3090006', []] in lines {'6'}
Station: ['23 ST', {'6'}, 'IRT', '405'

In [28]:
len(dict_stations)

108

In [29]:
df_station_list['Matched'].value_counts()

True     112
False     41
Name: Matched, dtype: int64

It matched 112 stations by using the exact name search. Let's examine the unmatched cases.

In [30]:
df_station_list[~df_station_list['Matched']].head()

,Station,Routes,Division,Station ID,Matched
0,LEXINGTON AV/59 ST,"{N, R, W}",BMT,7,False
16,WHITEHALL ST,"{R, W}",BMT,23,False
17,ESSEX ST,"{J, M, Z}",BMT,102,False
25,UNION SQ - 14 ST,{L},BMT,117,False
34,163 ST - AMSTERDAM AV,{C},IND,149,False


We need to manually find those unrecognized stations in the turntile data. The main reasons of failed matches include the format mismatch, different abbreviation, order of words, etc. Since there are only 41 of such items, a manual correction is feasible. That is part of the job for data science projects.

Here is an example of treating one case.

In [31]:
df_station_manual = df_station_list[~df_station_list['Matched']]
station_index = df_station_manual.index
iterId = 0
if iterId < len(station_index):
    print(f'Working on #{iterId} unmatched record with index: {station_index[iterId]}')
    print(f'The station name: {df_station_manual.iloc[iterId].Station}, routes: {df_station_manual.iloc[iterId].Routes}, div: {df_station_manual.iloc[iterId].Division}, Station ID: {df_station_manual.iloc[iterId]["Station ID"]}')   

Working on #0 unmatched record with index: 0
The station name: LEXINGTON AV/59 ST, routes: {'N', 'R', 'W'}, div: BMT, Station ID: 7


In [32]:
#df_station_manual["Station ID"].T.tolist()

In [33]:
# Check the unmatched station one after another and save the result into the manually input dictionary
keyword2search = 'LEXINGTON'
df_tt_list[df_tt_list['Station'].str.contains(keyword2search)]

,Station,Routes,Division,STATION_IDX,Geo_ID
209,LEXINGTON AV/53,"{6, M, E}",IND,t1062184,[]
273,LEXINGTON AV/63,{F},IND,t1069230,[]


In the turnstile data, there is no station with the name of "LEXINGTON AV/59 ST". Using the first keyword "LEXINGTON", it returns nothing. Then, try another keyword "59".

In [34]:
keyword2search = '59'
df_tt_list[df_tt_list['Station'].str.contains(keyword2search)]

,Station,Routes,Division,STATION_IDX,Geo_ID
0,59 ST,"{R, 4, Q, W, N, 5, 6}",BMT,t0000000,[400]
1,5 AV/59 ST,"{N, Q, R, W}",BMT,t0001001,[8]
50,59 ST,"{N, R, W}",BMT,t0004000,[]
146,59 ST COLUMBUS,"{C, A, 1, B, D}",IND,t1045133,[]
321,59 ST COLUMBUS,"{C, A, 1, B, D}",IRT,t3087133,[]
355,59 ST,"{R, 4, Q, W, N, 5, 6}",IRT,t3094000,[400]


By using another keyword "59", I can find a number of stations in which the third items in the list.

Here, I need to map the "STATION_IDX" value, i.e., "t0004000" in the turntile data to the "Station ID" value, i.e., "7", in the geolocation data.

I saved these manually identified pairs in a file, 'manual_map.txt'. 

In [35]:
# To facilitate the following processing, I further process the manually rendered mapping information and save the pairs into a csv file.
df_manual_match = pd.read_csv('manual_map.txt', skipinitialspace=True)

headers = ["turntile.station", "turntile.routes", "turntile.station_id", "geo.station", "geo.routes", "geo.station_id"]
rows2write = []
rows2write.append(headers)

for i in range(df_manual_match.shape[0]):
    geo_id = df_manual_match.at[i, "station_df_id"]
    index_tt = df_manual_match.at[i, "tt_df_id"]
    print('\n')
    print(f'Manually input station-geo pair #{i}')
    print(f'Turntile Station {df_tt_list.at[index_tt, "Station"]} w/ Routes: {df_tt_list.at[index_tt, "Routes"]}, Station ID: {df_tt_list.at[index_tt, "STATION_IDX"]}')
    print(f'Geo: {df_station_list.at[geo_id-1, "Station"]} w/ Routes: {df_station_list.at[geo_id-1, "Routes"]}')
    row = [df_tt_list.at[index_tt, "Station"], df_tt_list.at[index_tt, "Routes"], str(df_tt_list.at[index_tt, "STATION_IDX"]), \
          df_station_list.at[geo_id-1, "Station"], df_station_list.at[geo_id-1, "Routes"], str(df_station_list.at[geo_id-1, "Station ID"])]
    rows2write.append(row)

import csv
wrCSVfilename = 'turntile_station_map.csv'

with open(wrCSVfilename, mode='a', newline='') as rtd_file:
    csv_writer = csv.writer(rtd_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    for i in rows2write:
        #print(i[cols['frame_number']],i[cols['msgCopy']])
        csv_writer.writerow(i)
print('Writing Done!')



Manually input station-geo pair #0
Turntile Station 59 ST w/ Routes: {'N', 'R', 'W'}, Station ID: t0004000
Geo: LEXINGTON AV/59 ST w/ Routes: {'N', 'R', 'W'}


Manually input station-geo pair #1
Turntile Station WHITEHALL S - FRY w/ Routes: {'1', 'R', 'W'}, Station ID: t0008015
Geo: WHITEHALL ST w/ Routes: {'R', 'W'}


Manually input station-geo pair #2
Turntile Station DELANCEY/ESSEX w/ Routes: {'J', 'F', 'M', 'Z'}, Station ID: t0009016
Geo: ESSEX ST w/ Routes: {'J', 'M', 'Z'}


Manually input station-geo pair #3
Turntile Station 14 ST - UNION SQ w/ Routes: {'R', 'L', '4', 'Q', 'W', 'N', '5', '6'}, Station ID: t0005008
Geo: UNION SQ - 14 ST w/ Routes: {'L'}


Manually input station-geo pair #4
Turntile Station 163 ST - AMSTERDM w/ Routes: {'C'}, Station ID: t1041122
Geo: 163 ST - AMSTERDAM AV w/ Routes: {'C'}


Manually input station-geo pair #5
Turntile Station CATHEDRAL PKWY w/ Routes: {'C', 'B'}, Station ID: t1043128
Geo: CATHEDRAL PKWY (110 ST) w/ Routes: {'C', 'B'}


Manually i

In [36]:
df_manual_map = pd.read_csv('turntile_station_map.csv')
df_manual_map.head()
df_manual_map.shape

Next, I add these manually input station mappings into a dictionary along with the paired information obtained in the earlier automatic matching step.

In [40]:
for i in range(df_manual_map.shape[0]):
    if df_manual_map["turntile.station_id"][i] in dTest:
        dict_stations[df_manual_map["turntile.station_id"][i]].append(df_manual_map["geo.station_id"][i])
    else:
        dict_stations[df_manual_map["turntile.station_id"][i]] = [df_manual_map["geo.station_id"][i]]

In [41]:
#df_manual_map["turntile.station_id"][0]
len(dict_stations)

for i in dict_stations:
    print(f'There is a key: {i} w/ value of {dict_stations[i]}')

't0004000'

I have created a dictionary with the key:value pair of turntile_station_id:station_id. 

Next, I will use this dictionary and available information to create another dictionary to map the key of turntil_station_id to the station coordinates. As checked earlier, it is found that some station may be associated with multiple coordinates because it hosts multiple lines of services. In such a case, I will use the centroid of the point set instead. 

In [44]:
df_stations.head()

,index,Station ID,Division,Stop Name,Borough,Daytime Routes,GTFS Latitude,GTFS Longitude,Search ID
0,6,7,BMT,Lexington Av/59 St,M,N W R,40.762660,-73.967258,"Lexington Av/59 St,N W R,BMT,7"
1,7,8,BMT,5 Av/59 St,M,N W R,40.764811,-73.973347,"5 Av/59 St,N W R,BMT,8"
2,8,9,BMT,57 St - 7 Av,M,N Q R W,40.764664,-73.980658,"57 St - 7 Av,N Q R W,BMT,9"
3,9,10,BMT,49 St,M,N R W,40.759901,-73.984139,"49 St,N R W,BMT,10"
4,10,11,BMT,Times Sq - 42 St,M,N Q R W,40.754672,-73.986754,"Times Sq - 42 St,N Q R W,BMT,11"


In [45]:
# dict_stations
# df_stations

from statistics import mean

dict_loc = {}

for i in dict_stations:
    lat = df_stations[df_stations["Station ID"].isin(dict_stations[i])]["GTFS Latitude"].tolist()
    lat = mean(lat)
    lat = round(lat, 6)
    long = df_stations[df_stations["Station ID"].isin(dict_stations[i])]["GTFS Longitude"].tolist()
    long = mean(long)
    long = round(long, 6)
    print(f'Station: {i} has the coordinates: ({lat}, {long})')
    dict_loc[i] = (lat, long)

Station: t0001001 has the coordinates: (40.764811, -73.973347)
Station: t0001002 has the coordinates: (40.764664, -73.980658)
Station: t0001003 has the coordinates: (40.759901, -73.984139)
Station: t0002004 has the coordinates: (40.755983, -73.986229)
Station: t3085004 has the coordinates: (40.755983, -73.986229)
Station: t0003005 has the coordinates: (40.749719, -73.987823)
Station: t1003005 has the coordinates: (40.749719, -73.987823)
Station: t0004006 has the coordinates: (40.745494, -73.988691)
Station: t0004007 has the coordinates: (40.741303, -73.989344)
Station: t0005008 has the coordinates: (40.734789, -73.99073)
Station: t3092008 has the coordinates: (40.734673, -73.989951)
Station: t0004009 has the coordinates: (40.730328, -73.992629)
Station: t0004010 has the coordinates: (40.724329, -73.997702)
Station: t0006011 has the coordinates: (40.718803, -74.000193)
Station: t0004012 has the coordinates: (40.713282, -74.006978)
Station: t0007013 has the coordinates: (40.710668, -74.0

StatisticsError: mean requires at least one data point

In [46]:
dict_loc

{'t0001001': (40.764811, -73.973347),
 't0001002': (40.764664, -73.980658),
 't0001003': (40.759901, -73.984139),
 't0002004': (40.755983, -73.986229),
 't3085004': (40.755983, -73.986229),
 't0003005': (40.749719, -73.987823),
 't1003005': (40.749719, -73.987823),
 't0004006': (40.745494, -73.988691),
 't0004007': (40.741303, -73.989344),
 't0005008': (40.734789, -73.99073),
 't3092008': (40.734673, -73.989951),
 't0004009': (40.730328, -73.992629),
 't0004010': (40.724329, -73.997702),
 't0006011': (40.718803, -74.000193),
 't0004012': (40.713282, -74.006978),
 't0007013': (40.710668, -74.011029),
 't0004014': (40.70722, -74.013342),
 't0010017': (40.72028, -73.993915),
 't0011018': (40.713243, -74.003401),
 't0012019': (40.710368, -74.009509),
 't1012019': (40.710368, -74.009509),
 't1051019': (40.710368, -74.009509),
 't3051019': (40.710368, -74.009509),
 't0010020': (40.706476, -74.011056),
 't0029054': (40.739777, -74.002578),
 't0030069': (40.737335, -73.996786),
 't0031070': (4

In [101]:
df_tt_m = df_tt[df_tt.STATION_IDX.isin(dict_stations)].copy()  # make a copy of a slice of dataframe df_tt

# If df_tt_m is set using the get method below
# df_tt_m = df_tt[df_tt.STATION_IDX.isin(dict_stations)]
# The following changes on df_tt_m would generate warnings to alert about such change may affect the original dataframe
# It is because even pandas does not know df_tt_m is a copy or a view of df_tt
print(f'Turntile data in Manhattan has {df_tt_m.shape[0]} records compared to the total {df_tt.shape[0]} data records')

df_tt_m.head()

Turntile data in Manhattan has 89668 records compared to the total 206046 data records


,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,STATION_IDX,SEARCH_ID
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,10/26/2019,00:00:00,REGULAR,7247322,2455491,t0000000,"59 ST,NQR456W,BMT,t0000000"
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,10/26/2019,04:00:00,REGULAR,7247336,2455499,t0000000,"59 ST,NQR456W,BMT,t0000000"
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,10/26/2019,08:00:00,REGULAR,7247351,2455532,t0000000,"59 ST,NQR456W,BMT,t0000000"
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,10/26/2019,12:00:00,REGULAR,7247463,2455623,t0000000,"59 ST,NQR456W,BMT,t0000000"
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,10/26/2019,16:00:00,REGULAR,7247755,2455679,t0000000,"59 ST,NQR456W,BMT,t0000000"


In [102]:
#df_tt_m.loc[:,"LOCATION"] = df_tt_m.apply(lambda x: dict_loc[x["STATION_IDX"]], axis=1)

#df_tt_m.loc[:,"LOCATION"] = df_tt_m["STATION_IDX"].apply(lambda x: dict_loc[x])

df_tt_m["LOCATION"] = df_tt_m["STATION_IDX"].map(lambda x: dict_loc[x]) # Add the column saving the location information, i.e., (lat, lng)

In [103]:
df_grouped = df_tt_m.groupby(['UNIT', 'SCP', 'STATION', 'LINENAME', 'DIVISION'])  # divide rows into groups based on selected columns as an index 

# Use dataframe.diff() to calculate the difference between two consecutive rows regarding a specific column
# The first row has "NaN" values after calculation
df_tt_m['ENTRIES_DIFF']=df_grouped[['ENTRIES']].diff()
df_tt_m['EXITS_DIFF']=df_grouped[['EXITS']].diff()

df_tt_m = df_tt_m[~df_tt_m['ENTRIES_DIFF'].isnull()]  # remove all rows with 'NaN' in the 'ENTRIES_DIFF' column
df_tt_m.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,STATION_IDX,SEARCH_ID,LOCATION,ENTRIES_DIFF,EXITS_DIFF
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,10/26/2019,04:00:00,REGULAR,7247336,2455499,t0000000,"59 ST,NQR456W,BMT,t0000000","(40.762526, -73.967967)",14.0,8.0
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,10/26/2019,08:00:00,REGULAR,7247351,2455532,t0000000,"59 ST,NQR456W,BMT,t0000000","(40.762526, -73.967967)",15.0,33.0
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,10/26/2019,12:00:00,REGULAR,7247463,2455623,t0000000,"59 ST,NQR456W,BMT,t0000000","(40.762526, -73.967967)",112.0,91.0
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,10/26/2019,16:00:00,REGULAR,7247755,2455679,t0000000,"59 ST,NQR456W,BMT,t0000000","(40.762526, -73.967967)",292.0,56.0
5,A002,R051,02-00-00,59 ST,NQR456W,BMT,10/26/2019,20:00:00,REGULAR,7248159,2455733,t0000000,"59 ST,NQR456W,BMT,t0000000","(40.762526, -73.967967)",404.0,54.0


In [104]:
# Check the data
df_tt_m.describe()

,ENTRIES,EXITS,ENTRIES_DIFF,EXITS_DIFF
count,8.753300e+04,8.753300e+04,8.753300e+04,8.753300e+04
mean,4.610681e+07,4.070557e+07,2.838348e+03,1.283434e+03
std,2.219798e+08,2.159370e+08,7.957226e+05,3.411573e+05
min,0.000000e+00,0.000000e+00,-3.512054e+06,-4.146082e+06
25%,4.705220e+05,2.722620e+05,1.400000e+01,1.200000e+01
50%,2.651959e+06,1.704511e+06,9.700000e+01,7.700000e+01
75%,7.774998e+06,5.705378e+06,3.010000e+02,2.380000e+02
max,2.129010e+09,2.048818e+09,2.353883e+08,1.008450e+08


It is shown that the difference values may be negative which is largely due to the meter reading errors.

By checking the value in the time series, using a threshold of 5000 can detect the abnormal record.  

Then, it is also necessary to remove all records with negative difference values.

In [109]:
# First, check the difference with very large positive values
df_tt_m[df_tt_m.EXITS_DIFF>6000]

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,STATION_IDX,SEARCH_ID,LOCATION,ENTRIES_DIFF,EXITS_DIFF


In [110]:
df_tt_m = df_tt_m[df_tt_m.ENTRIES_DIFF<5000]
df_tt_m = df_tt_m[df_tt_m.EXITS_DIFF<6000]
df_tt_m = df_tt_m[df_tt_m.ENTRIES_DIFF>=0]
df_tt_m = df_tt_m[df_tt_m.EXITS_DIFF>=0]
df_tt_m.describe()

,ENTRIES,EXITS,ENTRIES_DIFF,EXITS_DIFF
count,8.652500e+04,8.652500e+04,86525.000000,86525.000000
mean,3.324924e+07,2.638846e+07,216.719700,186.553042
std,1.770056e+08,1.606036e+08,299.093053,294.503483
min,0.000000e+00,0.000000e+00,0.000000,0.000000
25%,4.596930e+05,2.642130e+05,15.000000,12.000000
50%,2.589164e+06,1.627667e+06,100.000000,78.000000
75%,7.522794e+06,5.388403e+06,304.000000,239.000000
max,2.115976e+09,2.038204e+09,3146.000000,5203.000000


In [111]:
df_tt_m.shape

(86525, 16)

[Back to the top](#top)
<a name="cleaned"></a>  
### Cleaned Data

Now, we've obtained a clean dataframe for the MTA turntile data. It contains 1) per turnstile entries and exits collected every four hours, 2) per station geolocation coordinates.

In [112]:
df_tt_m.head(5)

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,STATION_IDX,SEARCH_ID,LOCATION,ENTRIES_DIFF,EXITS_DIFF
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,10/26/2019,04:00:00,REGULAR,7247336,2455499,t0000000,"59 ST,NQR456W,BMT,t0000000","(40.762526, -73.967967)",14.0,8.0
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,10/26/2019,08:00:00,REGULAR,7247351,2455532,t0000000,"59 ST,NQR456W,BMT,t0000000","(40.762526, -73.967967)",15.0,33.0
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,10/26/2019,12:00:00,REGULAR,7247463,2455623,t0000000,"59 ST,NQR456W,BMT,t0000000","(40.762526, -73.967967)",112.0,91.0
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,10/26/2019,16:00:00,REGULAR,7247755,2455679,t0000000,"59 ST,NQR456W,BMT,t0000000","(40.762526, -73.967967)",292.0,56.0
5,A002,R051,02-00-00,59 ST,NQR456W,BMT,10/26/2019,20:00:00,REGULAR,7248159,2455733,t0000000,"59 ST,NQR456W,BMT,t0000000","(40.762526, -73.967967)",404.0,54.0


The cleaned data is saved into a csv for future analysis work.

In [113]:
df_tt_m.to_csv("modified_turntile_data.csv")